In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from data_process import TextEyeTrackingDataset
import numpy as np

# torch.manual_seed(seed)
# np.random.seed(seed)

# 加载数据
dataset = TextEyeTrackingDataset('data/skip_data.csv')
batch_size = 1

# 假设dataset是你已经加载的完整数据集
train_size = int(len(dataset) * 0.8)  # 以80%的数据作为训练集
test_size = len(dataset) - train_size  # 剩余20%的数据作为测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
import torch.nn as nn
from model import MultiModalModel
from train_and_evaluate import train_model
import warnings
from sklearn import exceptions

# 忽略特定类型的警告
warnings.filterwarnings("ignore", category=exceptions.UndefinedMetricWarning)


hidden_dim = 128  # LSTM隐藏层维度
output_dim = 1  # 输出维度，二分类问题

model = MultiModalModel(hidden_dim, output_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001)

pos_weight = torch.tensor([20])  # 大于1的值，表示正类相对于负类的权重
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)

tensorboard_dir = 'tensorlog/all'
# 训练模型
train_model(model, train_loader, test_loader, optimizer, criterion, device, tensorboard_dir, epochs=30)

KeyboardInterrupt: 

In [ ]:
!tensorboard --logdir='tensorlog/all'